#### Appending images 
http://apple.stackexchange.com/questions/52879/how-to-combine-two-images-into-one-on-a-mac
```
convert +append Wrong_0.png Wrong_1.png Wrong_2.png Wrong_3.png  Wrong.png
```

In [1]:
import pickle
import gzip
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as imgplot
import time
 #%matplotlib inline

In [5]:
#with open('Test.pickle') as f:
with open('Test_3Classes_NO_DMSO_Reviewer.pickle') as f:
    (pred_prob, Y_test, loss, cell_rows_test) = pickle.load(f)

In [6]:
pred = np.argmax(pred_prob, axis=1)

In [7]:
pred.max(), pred.min()

(2, 0)

In [8]:
#npzfile = np.load('HCS_Test_78x78.npz') #The original data
npzfile = np.load('HCS_Test_no_dmso_reviewer_78x78.npz')
cell_rows_test = npzfile['arr_0']
X_test = npzfile['arr_1']
#Y_test = npzfile['arr_2']
cell_rows_test.shape, X_test.shape, Y_test

((2223,), (2223, 5, 72, 72), array([0, 0, 0, ..., 2, 2, 2], dtype=int32))

In [8]:
X = np.zeros((5,5))
Y = np.ones((2,2))
X[1:3,1:3] = Y
X

array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  0.,  0.],
       [ 0.,  1.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [71]:
def plotRows(idx, filename='foo.png', figsize=(14,14), offset=0):
    fig = plt.figure(figsize=figsize)
    #fig = plt.figure()
    c = 0
    for i in idx:#range(X_idx.shape[0]):
        dd = X_test[i,:,:,:]
        ddd = np.transpose(dd.reshape(5*72,72))
        # Normalization of all channels
        for j in range(5):
            vals = ddd[:,(j*72):(j+1)*72]
            qmin = np.percentile(vals,0.01)
            qmax = np.percentile(vals,0.99)
            ddd[:,(j*72):(j+1)*72] = (ddd[:,(j*72):(j+1)*72] - vals.mean())/(vals.max() - vals.min())
        ax = fig.add_subplot(len(idx), 1, c+1)
        ax.set_axis_off()
        Y_true = Y_test[i]
        #ax.title("T={},P={}, {:3f}".format(Y_true, Y_pred_idx[i], pred_prob_idx[i, Y_true]),color='black')
        #ax.text(3,12,"T={},P={}, {:.2f}, {:.2f}".format(Y_true, pred[i], pred_prob[i, Y_true], pred_prob[i,pred[i]]),color='black', bbox={'facecolor':'white', 'alpha':0.7, 'pad':2})
        ax.text(3,12,"T={},P={}, {:.2f}".format(Y_true + offset, pred[i] + offset, pred_prob[i, Y_true]),color='black', bbox={'facecolor':'white', 'alpha':0.7, 'pad':2})
        #ax.imshow(np.log(ddd), interpolation='nearest',cmap=plt.get_cmap('Greys'))
        ax.imshow(-ddd, interpolation='nearest',cmap=plt.get_cmap('Greys'), )
        c += 1
    fig.subplots_adjust(hspace=0.0, wspace=-10.0)
    plt.savefig(filename, bbox_inches='tight')

In [76]:
np.sum(loss < 0.5) #For the alternative approach, we have 60 missclassified cells
idx_plot = np.where(loss < 0.5)[0]
sort_num = +(1 + Y_test[idx_plot]) * 100 + pred[idx_plot] * 10 + loss[idx_plot]

In [77]:
sort_num

array([ 110.        ,  110.00840048,  110.00000026,  110.22730219,
        110.00000004,  110.        ,  110.00000033,  110.00024491,
        110.        ,  110.00001816,  110.48399144,  110.        ,
        110.00048845,  110.30768335,  110.00587015,  110.02255269,
        110.1442149 ,  110.01052515,  110.00875333,  110.        ,
        110.01033576,  110.00720152,  110.00000384,  200.39684242,
        200.18245268,  200.25814766,  200.01003701,  200.00224748,
        200.00011931,  200.01875658,  200.10011734,  200.30588728,
        200.33656314,  200.03017233,  200.4866457 ,  200.41113231,
        200.11274619,  200.00311202,  200.05455409,  200.00000466,
        200.00002045,  300.0000027 ,  300.00507932,  310.0000029 ,
        110.00044734,  110.00236295,  110.00088876,  110.07637018,
        110.33603334,  200.03236751,  200.07137002,  200.00717266,
        120.00000018,  110.        ,  110.00090927,  200.09087134,
        200.00085842,  200.00000543,  200.25094056,  200.15182

In [78]:
d = np.argsort(sort_num)
idx_plot_sorted = np.zeros_like(idx_plot)
c = 0
for idx in d:
    idx_plot_sorted[c] = idx_plot[idx]
    c += 1

In [79]:
for i in range(4):
    plotRows(idx_plot_sorted[i*15:(i+1)*15], filename='Wrong_{}'.format(i), offset=1)

In [20]:
# Figure S1 Examples of phenotypes form cells
idx_0 = np.where(Y_test == 0)[0][10:15]
idx_1 = np.where(Y_test == 1)[0][90:95]
idx_2 = np.where(Y_test == 2)[0][10:15]
idx_3 = np.where(Y_test == 3)[0][6:11]
plotRows(np.append(idx_0, idx_1),filename='True_01.png')
plotRows(np.append(idx_2, idx_3),filename='True_23.png')

In [25]:
#np.where(Y_test == 0 and loss < 0.5)
#(loss < 0.5) and 
idx_0 = np.where((Y_test == 0) & (loss < 0.5))[0][0:5]
idx_1 = np.where((Y_test == 1) & (loss < 0.5))[0][0:5]
idx_2 = np.where((Y_test == 2) & (loss < 0.5))[0][0:9]
idx_3 = np.where((Y_test == 3) & (loss < 0.5))[0][0:1]
plotRows(np.append(idx_0, idx_1),filename='Wrong_01.png')
plotRows(np.append(idx_2, idx_3),filename='Wrong_23.png')